In [95]:
! hive --version

Hive 1.1.0
Subversion git://localhost.localdomain/Users/noland/workspaces/hive-apache/hive -r 3b87e226d9f2ff5d69385ed20704302cffefab21
Compiled by noland on Wed Feb 18 16:06:08 PST 2015
From source with checksum bca57a923a7578b7e5e9350ffb165cca


In [1]:
%%writefile creation_db.hql

DROP DATABASE IF EXISTS postsdb CASCADE;
CREATE DATABASE postsdb LOCATION '/user/jovyan/somemetastore';

Writing creation_db.hql


In [2]:
! hive -f creation_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.534 seconds
OK
Time taken: 0.291 seconds


<row Id="11" PostTypeId="1" AcceptedAnswerId="1248" CreationDate="2008-07-31T23:55:37.967" Score="1106" ViewCount="115048" Body="&lt;p&gt;Given a specific &lt;code&gt;DateTime&lt;/code&gt; value, how do I display relative time, like:&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;2 hours ago&lt;/li&gt;&#xA;&lt;li&gt;3 days ago&lt;/li&gt;&#xA;&lt;li&gt;a month ago&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" OwnerUserId="1" LastEditorUserId="1136709" LastEditorDisplayName="user2370523" LastEditDate="2015-12-29T02:08:37.450" LastActivityDate="2016-07-13T23:23:58.537" Title="How can relative time be calculated in C#?" Tags="&lt;c#&gt;&lt;datetime&gt;&lt;datediff&gt;&lt;relative-time-span&gt;" AnswerCount="33" CommentCount="4" FavoriteCount="508" CommunityOwnedDate="2009-09-04T13:15:59.820" />

In [37]:
%%writefile external_table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE postsdb;
DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external (
    rowid STRING,
    year STRING,
    month STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
 "input.regex" = '.*(?<=\\brow Id\\b=\")(\\d+).*(?<=\\bCreationDate\\b=\")(\\d{4})-(\\d{2}).*'
)
STORED AS TEXTFILE
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES ("skip.header.line.count"="1");

SELECT * FROM posts_sample_external LIMIT 10;

Overwriting external_table.hql


In [38]:
! hive -f external_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 0.49 seconds
OK
Time taken: 1.014 seconds
OK
Time taken: 0.362 seconds
OK
1394	2008	08
3543	2008	08
4521	2008	08
8689	2008	08
9062	2008	08
14671	2008	08
16307	2008	08
18780	2008	08
18929	2008	08
19668	2008	08
Time taken: 0.522 seconds, Fetched: 10 row(s)


In [100]:
%%writefile table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

SET hive.exec.dynamic.partition.mode=nonstrict;

USE postsdb;
DROP TABLE if exists posts_sample; 

CREATE TABLE posts_sample (
rowid int
) 
PARTITIONED BY (year INT, month INT);

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT rowid, year, month;

Overwriting table.hql


In [101]:
! hive -f table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 0.508 seconds
OK
Time taken: 3.023 seconds
OK
Time taken: 0.357 seconds
Query ID = jovyan_20180517072525_384a69b4-8293-4516-b1c2-12f7dd53b2a9
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1526536736924_0028, Tracking URL = http://fac3e30a4073:8088/proxy/application_1526536736924_0028/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526536736924_0028
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2018-05-17 07:25:25,144 Stage-1 map = 0%,  reduce = 0%
2018-05-17 07:25:41,807 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 14.48 sec
MapReduce Total cumulative CPU time: 14 

Partition postsdb.posts_sample{year=2010, month=06} stats: [numFiles=1, numRows=201, totalSize=1608, rawDataSize=1407]
Partition postsdb.posts_sample{year=2010, month=07} stats: [numFiles=1, numRows=209, totalSize=1672, rawDataSize=1463]
Partition postsdb.posts_sample{year=2010, month=08} stats: [numFiles=1, numRows=212, totalSize=1696, rawDataSize=1484]
Partition postsdb.posts_sample{year=2010, month=09} stats: [numFiles=1, numRows=208, totalSize=1664, rawDataSize=1456]
Partition postsdb.posts_sample{year=2010, month=10} stats: [numFiles=1, numRows=222, totalSize=1776, rawDataSize=1554]
Partition postsdb.posts_sample{year=2010, month=11} stats: [numFiles=1, numRows=236, totalSize=1888, rawDataSize=1652]
Partition postsdb.posts_sample{year=2010, month=12} stats: [numFiles=1, numRows=237, totalSize=1896, rawDataSize=1659]
Partition postsdb.posts_sample{year=2011, month=01} stats: [numFiles=1, numRows=276, totalSize=2208, rawDataSize=1932]
Partition postsdb.posts_sample{year=2011, month=

MapReduce Jobs Launched: 
Stage-Stage-1: Map: 1   Cumulative CPU: 14.74 sec   HDFS Read: 60003326 HDFS Write: 347401 SUCCESS
Total MapReduce CPU Time Spent: 14 seconds 740 msec
OK
Time taken: 38.38 seconds


In [132]:
%%writefile table.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE postsdb;

SELECT
    concat_ws("\t", year, concat_ws("-", year, month), lines_count)
FROM (
    SELECT
        string(year) as year
        ,string(month) as month
        ,string(lines_count) as lines_count
        ,ROW_NUMBER() OVER (ORDER BY year, month) as rownum
    FROM (
        SELECT year, month, COUNT(rowid) as lines_count
        FROM posts_sample
        GROUP BY year, month
    ) t1
) t2
WHERE rownum = 3;

Overwriting table.hql


In [133]:
! hive -f table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 0.571 seconds
Query ID = jovyan_20180517075353_64c8160f-aac0-43de-972f-8d522e35dc16
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526536736924_0049, Tracking URL = http://fac3e30a4073:8088/proxy/application_1526536736924_0049/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526536736924_0049
Hadoop job information for